# Applying ML to Predict Hotel Booking Cancellations

## The Project:
The hospitality industry relies heavily on accurate demand forecasting to optimize revenue and manage inventory. One significant challenge for hotels is the prediction of cancellations, which can disrupt business operations, lead to lost revenue, and impact customer satisfaction. Machine learning techniques offer a promising solution for predicting cancellations and mitigating their effects.

Hence, in this project, I aim to develop an ML model for hotel cancellation prediction, leveraging historical booking data and other relevant features. I will explore various algorithms and techniques, including logistic regression, random forest, gradient boosting, to identify the most effective approach for predicting cancellations. Ultimately, my goal is to provide a valuable tool for hotels to optimize their revenue management and improve customer satisfaction by accurately predicting and managing cancellations.

## The Dataset:

The dataset has been collected from <a href="https://www.sciencedirect.com/science/article/pii/S2352340918315191">ScienceDirect</a> and consists of hotel demand data for 2 hotels (H1 and H2) in Europe. Both datasets share the same structure, with 31 variables describing the 40,060 observations of H1 and 79,330 observations of H2. Each observation represents a hotel booking.